In [308]:

import xlwings as xw
import os
import pandas as pd
import numpy as np
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [309]:

#-- Batch 5A defined
batch5a = 'FSC-JAD-01-TEC-61-A1 MV FMECA-Batch 5 LBHD - AN.xlsx' 
pathBatch5a = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets'
shtBatch5a = '5- 4S WBT FMECA'
cellRangeBatch5a = 'A5:BP514'


Assigne the above variables to the ones used in this workbook

In [310]:
nameFile = batch5a
path = pathBatch5a
sheet = shtBatch5a
cellRange = cellRangeBatch5a

In [311]:
file_path = os.path.join(path,nameFile)
book = xw.Book(file_path)
sht = book.sheets[sheet]
rng = sht.range(cellRange)
df = rng.options(pd.DataFrame, omdex=False, header=True).value

In [312]:
df.shape



(509, 67)

### Stiffener problem

The stiffeners listed in the Vertech reports have some inconsistencies. <br>


Some appear as LL06-07 <br>

    In this case it will be replaced with the "nearest longitudinal stiffener". 
    It should be either one the above.
    
Some appear as HG1, HG2 etc. <br>

    HG1, HG2, HG3 are also the same as LL09, LL16, and LL22 respectively.
    They will be replaced with the stiffener names.

This will create a new column. <br>

The column should be coppied and replaced with the "stiffener" column in the FMECA sheet.


In [313]:
def stiffenerCorrection(row):
    stiffener = row['Stiffener(s)']
    nearest_lng = row['Nearest longitudinal member (SS,ObLBHD)']
    if stiffener == 'HG1':
        return 'LL09'
    elif stiffener == 'HG2':
        return 'LL16'
    elif stiffener == 'HG3':
        return 'LL22'
    elif len(stiffener.split('-')) == 2:
        return nearest_lng
    else:
        return stiffener

In [314]:
df['new_stiffener'] = df.apply(stiffenerCorrection,axis=1)

In [315]:
abt_lbhd = {"LL00":14,
"LL01":14,
"LL02":14,
"LL03":14,
"LL04":11.5,
"LL05":11.5,
"LL06":11.5,
"LL07":11.5,
"LL08":12,
"LL09":12,
"LL10":12,
"LL11":12,
"LL12":13,
"LL13":13,
"LL14":13,
"LL15":13,
"LL16":14.5,
"LL17":14.5,
"LL18":14.5,
"LL19":14.5,
"LL20":15,
"LL21":15,
"LL22":15,
"LL23":16,
"LL24":16,
"LL25":16,
"LL26":19.5,
"LL27":19.5,
"UD":20.5,
"BP":18.0
}

In [316]:
abt_web = {"LL00":10,
"LL01":10,
"LL02":10,
"LL03":10,
"LL04":10,
"LL05":10,
"LL06":10,
"LL07":10,
"LL08":10,
"LL09":10,
"LL10":10,
"LL11":10,
"LL12":10,
"LL13":10,
"LL14":10,
"LL15":10,
"LL16":10,
"LL17":10,
"LL18":10.5,
"LL19":10.5,
"LL20":10.5,
"LL21":10.5,
"LL22":10.5,
"LL23":10.5,
"LL24":10.5,
"LL25":10.5,
"LL26":10.5,
"LL27":10.5,
"UD":12.0,
"BP":13.5
}


In [317]:
abt_flange = {"LL00":15.5,
"LL01":15.5,
"LL02":17,
"LL03":17,
"LL04":18.5,
"LL05":18.5,
"LL06":17,
"LL07":17,
"LL08":20,
"LL09":20,
"LL10":20,
"LL11":20,
"LL12":20,
"LL13":20,
"LL14":22,
"LL15":22,
"LL16":19.5,
"LL17":19.5,
"LL18":18,
"LL19":18,
"LL20":20,
"LL21":21,
"LL22":23,
"LL23":23,
"LL24":19.5,
"LL25":20.5,
"LL26":24.5,
"LL27":14,
"UD":17,
"BP":25
}

In [318]:
def abt_check(row):
    
    if row['Local Structure'] == 'Plate':
        if row['Stiffener(s)'] in abt_lbhd:
            if abt_lbhd[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                return 'abt correct'
            else:
                return 'check abt'
        else:
            return 'find abt from drawings'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['Stiffener(s)'] in abt_web:
                if abt_web[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'
        elif row['Detail Structure'] == 'Flange':
            if row['Stiffener(s)'] in abt_flange:
                if abt_flange[row['Stiffener(s)']] == row["As Built Thickness\n(mm)"]:
                    return 'abt correct'
                else:
                    return 'check abt'
            else:
                return 'find abt from drawings'

        else:
            return 'not a web/flange'
    else:
        return 'not a plate/stiffener'
        

In [319]:
df['abtCheck'] = df.apply(abt_check, axis=1)

In [320]:
def correct_abt(row):
    vertech_abt = row["As Built Thickness\n(mm)"]
    if row['Local Structure'] == 'Plate':
        if row['abtCheck'] == 'check abt':
            return abt_lbhd[row['Stiffener(s)']]
        else:
            return vertech_abt
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            if row['abtCheck'] == 'check abt':
                return abt_web[row['Stiffener(s)']]
            else:
                return vertech_abt
        elif row['Detail Structure'] == 'Flange':
            if row['abtCheck'] == 'check abt':
                return abt_flange[row['Stiffener(s)']]
            else:
                return vertech_abt
    else:
        return vertech_abt
        

In [321]:

df['correctAbt'] = df.apply(correct_abt, axis=1)

In [322]:
correct_abts = df[['Anomaly ID','Stiffener(s)','As Built Thickness\n(mm)','Local Structure','Detail Structure','abtCheck','correctAbt','new_stiffener']]

In [323]:
correct_abts.shape

(509, 8)

#### As-built thickness corrected table

In order to view the table uncommate the line below.

In [324]:
#xw.view(correct_abts)

In [325]:
def finalAnomalyDescription(row):
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        return 'Pit within generalised corrosion'
    else:
        if row['Anomaly Nature'] == 'Corrosion':
            return 'Generalised corrosion'
        elif row['Anomaly Nature'] == 'Pitting':
            return 'Pit'
        else:
            return ''

In [326]:
df['final_anomlay_description'] = df.apply(finalAnomalyDescription,axis=1)

In [327]:
def locationStructure(row):
    if row['Local Structure'] == 'Plate':
        return 'Tank boundary plating'
    elif row['Local Structure'] == "Long'l stiff'r":
        if row['Detail Structure'] == 'Web':
            return 'Stiffener web'
        elif row['Detail Structure'] == 'Flange':
            return 'Stiffener flange middle 1/3'
    else:
        return ''

In [328]:
df['location_formulated'] = df.apply(locationStructure,axis=1)

In [329]:
def detailStatus(row):
    remaining_t = row['Average UTM Reading (mm)']
    loss = row['(mm)']
    renewable_loss = row["As Built Thickness\n(mm)"] - row["Renewal Thickness (mm)"]
    substantial_loss = row["As Built Thickness\n(mm)"] - row["Substantial Corer Thickness (mm)"]
    flag = row['FLAG']

    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if remaining_t > 10.0:
                if flag == "R":
                    return 'More than 10mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif 6.0 <= remaining_t and  remaining_t <= 10.0:
                if flag == "R":
                    return 'Between 6 - 10mm remaining'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange middle 1/3':
            if loss > renewable_loss:
                if remaining_t < 6.0:
                    return 'Any loss greater than renewable (<6mm)'
                else:
                    return 'Any loss greater than renewable'
            else:
                return "Substantial or below substantial loss"
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if remaining_t >= 6.0:
                if flag == "R":
                    return 'More than 6mm remaining and greater than renewable loss'
                else:
                    return "Substantial or below substantial loss"
            elif remaining_t < 6.0:
                return 'Less than 6mm remaining'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if remaining_t < 6.0:
            return 'Less than 6mm remaining'
        else:
            return "Greater than 6mm remaining"


    

In [330]:
def repairMethod(row):
    detail = row['Detail']
    if row['Final Anomaly description'] == 'Pit':
        if row['Location'] == 'Tank boundary plating':
            if detail == 'More than 10mm remaining and greater than renewable loss':
                return 'investigate'
            elif detail == 'Between 6 - 10mm remaining':
                return 'Arrest and monitor CVI'
            elif detail == 'Less than 6mm remaining':
                return 'Permanent repair'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener web':
            if detail == 'Any loss greater than renewable':
                return 'investigate coating condition'
            else:
                return 'investigate coating condition'
        if row['Location'] == 'Stiffener flange middle 1/3':
            if detail == 'Any loss greater than renewable':
                return 'Arrest and monitor CVI'
            else:
                return 'investigate'
        if row['Location'] == 'Stiffener flange outside of middle 1/3':
            if detail == 'More than 6mm remaining and greater than renewable loss':
                return 'A&M CVI or investigate'
            elif detail == 'Less than 6mm remaining':
                return 'investigate'
    if row['Final Anomaly description'] == 'Generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'
    if row['Final Anomaly description'] == 'Pit within generalised corrosion':
        if detail == 'More than 10mm remaining and greater than renewable loss':
            return 'investigate'
        elif detail == 'Between 6 - 10mm remaining':
            return 'Arrest and monitor CVI'
        elif detail == 'Less than 6mm remaining':
            return 'Permanent repair'
        else:
            return 'investigate'

In [331]:
def RepairMethodR1(row):
    detail = row['Detail']
    coating = row['Coating condition']
    anomaly_type = row['Final Anomaly description']
    location = row['Location']

    if coating != "":
        if anomaly_type == 'Pit':
            if location == 'Tank boundary plating':
                if detail == 'More than 10mm remaining and greater than renewable loss' or detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                elif detail == 'Between 6 - 10mm remaining':
                    return 'Arrest and monitor CVI'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
                elif detail == "Substantial or below substantial loss":
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
                else:
                    return 'Pit. not above 10 or below 10?'

            elif location == 'Stiffener web':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange middle 1/3':
                if detail == 'Any loss greater than renewable':
                    return "Arrest and monitor CVI"
                elif detail == 'Any loss greater than renewable (<6mm)':
                    return 'Permanent repair'
                else:
                    if coating == 'Fair' or coating == 'Poor':
                        return "Arrest and monitor CVI"
                    else:
                        return "Monitor only, no coating"
            elif location == 'Stiffener flange outside of middle 1/3':
                if detail == 'More than 6mm remaining and greater than renewable loss':
                    return 'Permanent repair'
                elif detail == 'Less than 6mm remaining':
                    return 'Permanent repair'
        if anomaly_type == 'Generalised corrosion':
            
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
        if anomaly_type == 'Pit within generalised corrosion':
            if detail == 'Less than 6mm remaining':
                return 'Structural repair'
            elif detail == "Greater than 6mm remaining":
                if coating == 'Fair' or coating == 'Poor':
                    return "Arrest and monitor CVI"
                else:
                    return "Monitor only (not part of standard repair)"
            else:
                return 'Corr. not above 10, or below 10?'
    else:
        return 'define coating condition'


In [332]:
df['detail_formulated'] = df.apply(detailStatus,axis=1)

In [333]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining


In [334]:
dm = 'decisionMatrix.xlsx' 
pathANworking = 'C:/Users/nisha/OneDrive - Floating Solutions Consulting/Documents/04. Projects/JAD-01 MV Main Deck/03. Working/FMECA/Batch 5 FMECA 2022/FMECA spreadsheets/AN_working'
decisionMatrixPath = os.path.join(pathANworking,dm)

In [346]:
dimLevels = pd.read_excel(decisionMatrixPath,sheet_name='diminutionLevels')

In [335]:
dm = pd.read_excel(decisionMatrixPath,sheet_name='decisionMatrix')

In [347]:
dimLevels

,Primary structure,Local structure,Diminution level
0,Long'l bhd,Plate,0.20
1,Long'l bhd,Long'l stiff'r,0.25


In [336]:
dm

,Anomaly type,Location,Detail,Repair,Inspection interval
0,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Permanent repair,Scheduled tank inspection
1,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Temporary repair,1 year λ
2,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,Arrest and monitor CVI,2.5 years
3,Pit,Tank boundary plating,More than 10mm remaining and greater than rene...,"Monitor only, no coating",1 year
4,Pit,Tank boundary plating,Between 6 - 10mm remaining,Permanent repair,Scheduled tank inspection
5,Pit,Tank boundary plating,Between 6 - 10mm remaining,Temporary repair,1 year λ
6,Pit,Tank boundary plating,Between 6 - 10mm remaining,Arrest and monitor CVI,1 year λ
7,Pit,Tank boundary plating,Less than 6mm remaining,Permanent repair,Scheduled tank inspection
8,Pit,Tank boundary plating,Less than 6mm remaining,Temporary repair,1 year
9,Pit,Tank boundary plating,Substantial or below substantial loss,Arrest and monitor CVI,1 year λ


In [338]:
def inspectionInterval(row):
    anomaly_type = row['Final Anomaly description']
    location = row['Location']
    detail = row['Detail']
    repair = row['Repair']

    if anomaly_type == 'Pit':
        a = dm[dm['Anomaly type']==anomaly_type]
        b = a[a['Location']==location]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'
    else:
        b = dm[dm['Anomaly type']==anomaly_type]
        c = b[b['Detail']==detail]
        d = c[c['Repair'] == repair]
        try:
            interval = d.iloc[0,-1]
        except:
            interval = 'error'

    return interval
    
    



In [339]:
df['inspInt_formulate'] = df.apply(inspectionInterval,axis=1)

In [369]:
def allowablePercentage(row):

    primary_structure = row["Primary Structure"]
    local_structure = row["Local Structure"]

    a = dimLevels[dimLevels['Primary structure']==primary_structure]
    b = a[a['Local structure']==local_structure]
    try:
        allowable = b.iloc[0,-1]
    except:
        allowable = float(0)

    return allowable


In [375]:
def allowablechecks(row):
    dimLevelVertech = float(row["Max Allowable Diminution (%)"])
    dimLevelFSC = float(row["allowables"])
    diff = np.abs(dimLevelVertech-dimLevelFSC)
    if diff == 0:
        return 'correct dim level'
    else:
        return 'check dim level'



In [376]:
df['allowables'] = df.apply(allowablePercentage,axis=1)

In [377]:
df['allowableCheck'] = df.apply(allowablechecks,axis=1)

In [378]:
df.head(3)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,inspInt_formulate,repair_formulate,allowables,allowableCheck
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.7900,3.7,6.2100,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,-,Arrest and monitor CVI,0.2,correct dim level
2.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-193,AW,49-50,LL21,LBHD,4000.0,100.0,FR.49,50.0,LL21,0.0,AH,48.7,15.0,0.2,12.0,12.75,6.9,5.8,4.1,8.5,10.8,11.8,12.1,12.1,NaN,NaN,9.0125,4.1,5.9875,0.399167,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,5.0,100.0,0.399167,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,None,Poor,LL21,abt correct,15.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,-,Arrest and monitor CVI,0.2,correct dim level
3.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-194,AW,50-51,LL11,LBHD,15.0,15.0,FR.51,N/A,LL11,N/A,AH,0.0,12.0,0.2,9.6,10.20,5.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.5000,5.5,6.5000,0.541667,R,Long'l bhd,Panel,Plate,None,Pitting,Long'l bhdPanelPlatePitting,8.0,7.0,8.0,448.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,8.0,3.0,4.0,96.0,NaN,0.0,0.541667,Local,Med,Med,N,Pit,Tank boundary plating,Less than 6mm remaining,Permanent repair,None,None,Structural repair (Pad weld),Good,LL11,abt correct,12.0,Pit,Tank boundary plating,Less than 6mm remaining,Scheduled tank inspection,Permanent repair,0.2,correct dim level


In [340]:
df['repair_formulate'] = df.apply(RepairMethodR1,axis=1)

In [341]:
df.head(1)

,Report reference,Anomaly ID,Anomaly Type,Frame(s),Stiffener(s),Structural Component,Length (X),Width/Height\n(Y),"Nearest transverse member (Fr., TWF,TBHD)",Longitudinal\n(l) (distance\nfrom in mm),"Nearest longitudinal member (SS,ObLBHD)",Transverse (t) (distance from in mm),Grade,Weight\n(kgs),As Built Thickness\n(mm),Max Allowable Diminution (%),Renewal Thickness (mm),Substantial Corer Thickness (mm),EN01,EN02,EN03,EN04,EN05,EN06,EN07,EN08,EN09,EN10,Average UTM Reading (mm),Minimum \nUTT \nReading,(mm),(%),FLAG,Primary Structure,Secondary Structure,Local Structure,Detail Structure,Anomaly Nature,Concatenate,Sev,Occ,Det,RPN\npre-action,Adj Occ pre,Adj\nRPN \npre-action,Required Engineering Action,Has engineering action been completed ?,Generic action (following engineering action),Sev2,Occ2,Det2,RPN\npost-action,Adj Occ post,Adj\nRPN \npost-action,Avg % diminution,Anom extents,Stress level (location on span),Proximity / density of anom.,Stress / buckling calc?,Final Anomaly description,Location,Detail,Repair,Inspection interval,Assessment notes,Adj Final action,Coating condition,new_stiffener,abtCheck,correctAbt,final_anomlay_description,location_formulated,detail_formulated,inspInt_formulate,repair_formulate
Item No.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.0,VER-52531-4SWBT-LBHD-CR-0001-1379,4SWBT-LBHD- CR-192,AW,49-50,LL15,LBHD,4200.0,100.0,FR.49,500.0,LL15,0.0,AH,44.5,13.0,0.2,10.4,11.05,8.0,6.9,8.7,3.7,4.1,7.5,7.9,7.4,7.5,6.2,6.79,3.7,6.21,0.477692,R,Long'l bhd,Panel,Plate,None,Corrosion,Long'l bhdPanelPlateCorrosion,5.0,7.0,8.0,280.0,None,0.0,Investigate further and perform calculation if...,No,Arrest & monitor CVI,5.0,3.0,4.0,60.0,NaN,0.0,0.477692,Local,Med,Med,Y,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,Arrest and monitor CVI,None,None,None,Poor,LL15,abt correct,13.0,Generalised corrosion,Tank boundary plating,Greater than 6mm remaining,-,Arrest and monitor CVI


In [342]:
summary_df_cols = ['Report reference', 'Anomaly ID','Stiffener(s)','Nearest longitudinal member (SS,ObLBHD)',
       'As Built Thickness\n(mm)', 
       'Renewal Thickness (mm)', 'Substantial Corer Thickness (mm)',
       'Average UTM Reading (mm)', 'Minimum \nUTT \nReading ', '(mm)', '(%)',
       'FLAG','correctAbt',
       'final_anomlay_description',
       'location_formulated', 'detail_formulated', 'repair_formulate']

In [343]:
#xw.view(df[summary_df_cols])

In [345]:
xw.view(df)

In [ ]:
#xw.view(df.groupby(['repair_formulate']).count())

In [ ]:
#df.groupby(['repair_formulate']).count().reset_index()

In [ ]:
inv_df = df[df['Repair']=='investigate']

In [ ]:
inv_df.head(1)

In [ ]:
inv_df.groupby(['Location','Detail','Repair','Coating condition']).agg({'Repair':'count'})